# Hands-on: Evaluating AI-Generated Text: Automatic Metrics vs. Human Judgments

In real-world applications like news aggregation, research briefs, or internal company reports, AI models can automatically generate summaries to save time and reduce human effort.
In this lab, we explore how to evaluate AI-generated summaries using automatic metrics and compare these evaluations with human judgments. The objective is to understand not only whether the generated text is relevant, fluent, and coherent, but also how closely automatic evaluation aligns with human intuition.

**What You'll Learn:**
- How to evaluate AI-generated summaries using automatic metrics.
- How to compare AI scores with human judgments for relevance and quality.
- How to analyze results to assess alignment between AI and human evaluation.

## Step 1: Define the Task

Choose a sample text generation task.
Example:
- **Task:** Summarization
- **Input:** Short news article
- **Expected output:** Concise, accurate summary

In [1]:
input_texts = [
    "Researchers at a leading technology lab have developed an advanced AI system capable of generating highly realistic 3D models from simple 2D sketches. This breakthrough has the potential to transform workflows in the gaming and film industries, where designers can now quickly prototype complex scenes. The AI uses a combination of generative modeling techniques and neural rendering, allowing it to understand depth, texture, and perspective from minimal input. Researchers believe this tool could drastically reduce the time and cost associated with creating 3D assets while democratizing access for smaller studios and independent creators.",
    "A new reinforcement learning algorithm has been introduced that allows autonomous robots to navigate extremely complex and dynamic environments with very little human intervention. The algorithm combines advanced path planning, obstacle avoidance, and real-time decision-making capabilities. It has been tested in indoor and outdoor scenarios, including cluttered warehouse environments and uneven terrain. Engineers report that the system adapts efficiently to unforeseen obstacles, learns from its interactions with the environment, and achieves high success rates in completing navigation tasks without supervision.",
    "Deep learning engineers have designed a predictive maintenance model that analyzes sensor data from industrial equipment to forecast potential failures before they occur. By using historical data, real-time telemetry, and anomaly detection algorithms, the model helps plant operators anticipate breakdowns, schedule maintenance, and optimize resource allocation. In trials across several manufacturing facilities, the AI system successfully reduced downtime, improved operational efficiency, and enhanced safety by identifying equipment at risk before a catastrophic failure.",
    "AI-powered natural language processing tools have advanced to the point where they can accurately summarize lengthy legal documents, including contracts, court filings, and regulatory paperwork. These tools use transformer-based architectures to extract key clauses, summarize obligations, and highlight potential risks. Law firms and corporate legal departments are increasingly adopting these tools to reduce the time spent reviewing documents, improve accuracy, and allow legal professionals to focus on higher-value analytical tasks. The technology has the potential to significantly streamline legal workflows and increase access to legal insights.",
    "A team of AI researchers has developed a machine learning model capable of detecting deepfake videos in real time, an emerging threat in the age of digital misinformation. The system uses a combination of computer vision, temporal analysis, and neural network classifiers to identify subtle inconsistencies in video frames that indicate manipulation. Tested across a large dataset of authentic and manipulated videos, the AI achieved high accuracy, even under challenging conditions such as varying lighting and compression artifacts. This technology is intended to help social media platforms, news organizations, and government agencies combat the spread of deepfake content and protect information integrity."
]

reference_summaries = [
    "Tech lab develops AI that converts sketches into realistic 3D models for games and movies, reducing production time and cost.",
    "New reinforcement learning algorithm allows robots to navigate complex spaces autonomously, adapting to dynamic obstacles.",
    "Deep learning model predicts industrial equipment failures to reduce downtime, improve safety, and optimize maintenance.",
    "AI NLP tools summarize legal documents, saving lawyers time and improving accuracy in reviewing contracts and filings.",
    "AI model detects deepfake videos in real time to fight misinformation, achieving high accuracy across diverse datasets."
]



In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Qwen model locally
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
generated_summaries = []

for article in input_texts:
    prompt = f"Please read the following article and generate a concise one sentence summary:\n\n{article}"

    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate summary
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=150
    )

    # Remove input prompt from generated output
    generated_ids_trimmed = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    summary = tokenizer.batch_decode(generated_ids_trimmed, skip_special_tokens=True)[0]

    generated_summaries.append(summary)

# Verify first generated summary
print("Generated Summary for first article:\n", generated_summaries[0])


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Generated Summary for first article:
 A new AI system developed by researchers can create highly realistic 3D models from simple 2D sketches, potentially revolutionizing workflows in gaming and film industries.


In [3]:
for summary in generated_summaries:
  print(summary)

A new AI system developed by researchers can create highly realistic 3D models from simple 2D sketches, potentially revolutionizing workflows in gaming and film industries.
An innovative reinforcement learning-based robot navigation algorithm enables autonomous machines to navigate complex, dynamic environments effectively with minimal human intervention.
A predictive maintenance model based on deep learning analyzes sensor data from industrial equipment to forecast potential failures, reducing downtime and improving operational efficiency.
AI-powered natural language processing tools now accurately summarize lengthy legal documents using transformer-based architectures, enabling law firms and corporate legal departments to save time and enhance efficiency while focusing on more critical tasks.
An AI-powered machine learning model can detect deepfake videos in real-time with high accuracy using computer vision, temporal analysis, and neural network classifiers.


## Step 2: Define Evaluation Rubric

Human evaluators will rate outputs on a scale from 1–5 based on these dimensions:

| Criterion       | Description                                    | Scale |
| --------------- | ---------------------------------------------- | ----- |
| **Fluency**     | Is the text grammatically correct and natural? | 1–5   |
| **Relevance**   | Does it contain the key information?           | 1–5   |
| **Coherence**   | Are the ideas logically connected?             | 1–5   |
| **Conciseness** | Is it succinct without missing key info?       | 1–5   |
| **Overall**     | General quality of the output                  | 1–5   |


## Step 3: Collect Human Annotations

Create a small dataset of generated outputs and have humans score them:

In [ ]:
# Human annotations with diverse scores to demonstrate evaluation differences
# Note: In practice, these would come from multiple human evaluators
human_annotations = [
    # 1. 3D models AI - Human thinks it's very good but slightly verbose
    {"fluency": 5, "relevance": 5, "coherence": 5, "conciseness": 4, "overall": 5},

    # 2. Reinforcement learning robots - Human finds it good but a bit technical/wordy
    {"fluency": 4, "relevance": 4, "coherence": 4, "conciseness": 3, "overall": 4},

    # 3. Deep learning predictive maintenance - Human rates it as excellent
    {"fluency": 5, "relevance": 5, "coherence": 5, "conciseness": 5, "overall": 5},

    # 4. NLP legal document summarization - Human thinks it's too long, loses conciseness
    {"fluency": 5, "relevance": 4, "coherence": 5, "conciseness": 3, "overall": 4},

    # 5. Deepfake detection AI - Human rates it highly
    {"fluency": 5, "relevance": 5, "coherence": 5, "conciseness": 4, "overall": 5},
]


## Step 4: Automatic Evaluation
We can use multiple automatic metrics:

In [5]:
!pip install rouge-score nltk bert-score --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00


### **ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**

ROUGE is a set of metrics that compare overlap of n-grams, sequences, and word pairs between a generated text and a reference text.

**Common variants:**

- ROUGE-1: Overlap of unigrams (single words)

- ROUGE-L: Longest common subsequence (measures sentence-level structure)

**Interpretation:** Higher ROUGE F1 score → generated text covers more of the reference content.

In [6]:
from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge_scores_list = []

for ref, gen in zip(reference_summaries, generated_summaries):
    rouge_scores = scorer.score(ref, gen)

    rouge_f1 = (
        rouge_scores['rouge1'].fmeasure +
        rouge_scores['rougeL'].fmeasure
    ) / 2

    rouge_scores_list.append(rouge_f1)

    print(f"Generated: {gen}")
    print(
        f"ROUGE-1 F1: {rouge_scores['rouge1'].fmeasure:.3f}, "
        f"ROUGE-L F1: {rouge_scores['rougeL'].fmeasure:.3f}, "
        f"Combined ROUGE F1: {rouge_f1:.3f}"
    )
    print("-"*60)

# 🔹 Overall ROUGE score
overall_rouge = np.mean(rouge_scores_list)
print(f"\nOverall ROUGE F1 score: {overall_rouge:.3f}")


Generated: A new AI system developed by researchers can create highly realistic 3D models from simple 2D sketches, potentially revolutionizing workflows in gaming and film industries.
ROUGE-1 F1: 0.356, ROUGE-L F1: 0.267, Combined ROUGE F1: 0.311
------------------------------------------------------------
Generated: An innovative reinforcement learning-based robot navigation algorithm enables autonomous machines to navigate complex, dynamic environments effectively with minimal human intervention.
ROUGE-1 F1: 0.500, ROUGE-L F1: 0.389, Combined ROUGE F1: 0.444
------------------------------------------------------------
Generated: A predictive maintenance model based on deep learning analyzes sensor data from industrial equipment to forecast potential failures, reducing downtime and improving operational efficiency.
ROUGE-1 F1: 0.667, ROUGE-L F1: 0.410, Combined ROUGE F1: 0.538
------------------------------------------------------------
Generated: AI-powered natural language processin

A high ROUGE F1 score  indicates that the generated summaries preserve the main content and structure of the references, despite limited exact phrasing overlap.

### **BLEU (Bilingual Evaluation Understudy)**

BLEU measures precision of n-gram overlap between generated text and reference text.

- Focuses on how many words/phrases in the generated text appear in the reference.

- Usually BLEU ranges from 0 to 1, with higher values meaning the output is closer to the reference.

- Often used in machine translation and text generation evaluation.

In [7]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

bleu_scores_list = []
smooth = SmoothingFunction().method1

for ref, gen in zip(reference_summaries, generated_summaries):
    reference_tokens = [ref.split()]
    candidate_tokens = gen.split()

    bleu_score_val = sentence_bleu(
        reference_tokens,
        candidate_tokens,
        smoothing_function=smooth
    )

    bleu_scores_list.append(bleu_score_val)
    print(f"Generated: {gen}")
    print(f"BLEU score: {bleu_score_val:.3f}")
    print("-"*60)

# 🔹 Overall BLEU score
overall_bleu = np.mean(bleu_scores_list)
print(f"\nOverall BLEU score: {overall_bleu:.3f}")

Generated: A new AI system developed by researchers can create highly realistic 3D models from simple 2D sketches, potentially revolutionizing workflows in gaming and film industries.
BLEU score: 0.043
------------------------------------------------------------
Generated: An innovative reinforcement learning-based robot navigation algorithm enables autonomous machines to navigate complex, dynamic environments effectively with minimal human intervention.
BLEU score: 0.026
------------------------------------------------------------
Generated: A predictive maintenance model based on deep learning analyzes sensor data from industrial equipment to forecast potential failures, reducing downtime and improving operational efficiency.
BLEU score: 0.022
------------------------------------------------------------
Generated: AI-powered natural language processing tools now accurately summarize lengthy legal documents using transformer-based architectures, enabling law firms and corporate legal 

An low overall BLEU score indicates minimal exact n-gram overlap between generated and reference summaries and highlights BLEU’s limitations for evaluating paraphrased LLM outputs.

### **AI-as-a-Judge**
AI as a judge is the use of a language model to automatically evaluate the quality of generated text.

In this setup, we will use the Gemini API to assess summaries and check their relevance against the ground truth, providing scores and justifications that can be compared with human evaluations.

In [8]:
from openai import OpenAI
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

In [9]:
import re
import json
import pandas as pd

In [10]:
# Ask AI to act as a judge and score the summary
judge_prompt_template = """
Your task is to score the relevance between a generated answer and the question, based on the ground truth answer, in the range 1 to 5. Also, provide a short justification for your score. Your primary focus should be on determining whether the generated answer contains sufficient and correct information to address the given question according to the ground truth answer. If the generated answer contradicts the ground truth, it should receive a low score (1–2). If it partially addresses the question but misses important details, it should receive a moderate score (3–4). If it is fully consistent and complete, it should receive a high score (5).

Scoring Scale:
1 = Completely irrelevant or contradicts the ground truth
2 = Mostly irrelevant or only tangentially related
3 = Somewhat relevant but incomplete or missing major points
4 = Mostly relevant, with minor omissions or inaccuracies
5 = Fully relevant and consistent with the ground truth

Evaluate the RELEVANCE of the generated answer to the question, considering the provided ground truth answer.

Question:
{{question}}

Ground Truth Answer:
{{ground_truth}}

Generated Answer:
{{generated_answer}}

Output your response in valid JSON format:
{
  "relevance": <integer between 1 and 5>,
  "justification": "<brief explanation>"
}

Example:
Question: Is the sky blue?
Ground Truth Answer: Yes, the sky is blue.
Generated Answer: No, the sky is not blue.
Expected Output:
{
  "relevance": 1,
  "justification": "The generated answer contradicts the ground truth; it incorrectly claims the sky is not blue."
}
"""
evaluation_results = []

for i, (gen, ref, article) in enumerate(zip(generated_summaries, reference_summaries, input_texts)):
    # Include the article in the question
    question_text = f"Summarize the following article accurately in one sentence:\n\n{article}"

    # Fill in placeholders
    eval_prompt = judge_prompt_template.replace("{{ground_truth}}", ref)\
                                       .replace("{{generated_answer}}", gen)\
                                       .replace("{{question}}", question_text)

    # Send request to Gemini judge
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert evaluator scoring summary relevance."},
            {"role": "user", "content": eval_prompt}
        ],
        temperature=0
    )

    result = response.choices[0].message.content.strip()
    evaluation_results.append(result)
    print(f"Evaluated Summary {i+1}:\n", result, "\n")

# === Parse JSON results robustly ===
parsed_results = []
for res in evaluation_results:
    # Extract JSON block
    match = re.search(r"\{.*\}", res, re.DOTALL)
    if match:
        try:
            parsed_json = json.loads(match.group())
            # Ensure relevance is an int
            if parsed_json.get("relevance") is not None:
                parsed_json["relevance"] = int(parsed_json["relevance"])
            parsed_results.append(parsed_json)
        except json.JSONDecodeError:
            parsed_results.append({"relevance": None, "justification": res})
    else:
        parsed_results.append({"relevance": None, "justification": res})

# Build the evaluation DataFrame
df_eval = pd.DataFrame({
    "Generated Summary": generated_summaries,
    "Reference Summary": reference_summaries,
    "Relevance Score": [r["relevance"] for r in parsed_results],
    "Justification": [r["justification"] for r in parsed_results]
})

df_eval.head()


Evaluated Summary 1:
 ```json
{
  "relevance": 5,
  "justification": "The generated answer accurately summarizes the article, capturing the key points about the AI system, its capabilities, and its potential impact on the gaming and film industries. It aligns well with the ground truth answer."
}
``` 

Evaluated Summary 2:
 ```json
{
  "relevance": 5,
  "justification": "The generated answer accurately summarizes the article in one sentence, capturing the key aspects of the new reinforcement learning algorithm and its capabilities."
}
``` 

Evaluated Summary 3:
 ```json
{
  "relevance": 5,
  "justification": "The generated answer accurately summarizes the article and aligns with the ground truth, capturing the core idea of a deep learning model predicting equipment failures to improve efficiency and reduce downtime."
}
``` 

Evaluated Summary 4:
 ```json
{
  "relevance": 5,
  "justification": "The generated answer accurately summarizes the article, capturing the key points about AI NLP

,Generated Summary,Reference Summary,Relevance Score,Justification
0,A new AI system developed by researchers can c...,Tech lab develops AI that converts sketches in...,5,The generated answer accurately summarizes the...
1,An innovative reinforcement learning-based rob...,New reinforcement learning algorithm allows ro...,5,The generated answer accurately summarizes the...
2,A predictive maintenance model based on deep l...,Deep learning model predicts industrial equipm...,5,The generated answer accurately summarizes the...
3,AI-powered natural language processing tools n...,"AI NLP tools summarize legal documents, saving...",5,The generated answer accurately summarizes the...
4,An AI-powered machine learning model can detec...,AI model detects deepfake videos in real time ...,5,The generated answer accurately summarizes the...


In [ ]:
# Install visualization libraries if needed
%pip install matplotlib seaborn scipy --quiet

## Step 5: Comparing AI and Human Evaluation

To assess how well the AI judge aligns with human judgment, we compare the relevance scores assigned by the AI with those provided by human annotators. Each generated summary is scored for how accurately it captures the main points of the original article. By examining the correlation and differences between AI and human relevance scores, we can measure the AI’s reliability and consistency in evaluating text quality. This comparison helps validate the automatic evaluation process and ensures it approximates human-level assessment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Human-annotated relevance scores
human_relevance = [h["relevance"] for h in human_annotations]

# AI judge relevance scores (from parsed_results)
ai_relevance = [r["relevance"] for r in parsed_results]

# Build a comprehensive comparison DataFrame
df_relevance = pd.DataFrame({
    "Summary_ID": range(1, len(human_relevance) + 1),
    "Human_Relevance": human_relevance,
    "AI_Relevance": ai_relevance,
    "Generated_Summary": generated_summaries,
    "Reference_Summary": reference_summaries
})

# Compute absolute differences
df_relevance["Absolute_Difference"] = abs(df_relevance["Human_Relevance"] - df_relevance["AI_Relevance"])
df_relevance["Agreement"] = df_relevance["Absolute_Difference"] == 0

# Calculate correlation metrics
pearson_corr, pearson_p = pearsonr(human_relevance, ai_relevance)
spearman_corr, spearman_p = spearmanr(human_relevance, ai_relevance)

# Calculate agreement metrics
exact_agreement = (df_relevance["Absolute_Difference"] == 0).sum()
within_one = (df_relevance["Absolute_Difference"] <= 1).sum()
mean_absolute_error = df_relevance["Absolute_Difference"].mean()
max_difference = df_relevance["Absolute_Difference"].max()

print("="*80)
print("HUMAN vs AI RELEVANCE SCORE COMPARISON")
print("="*80)
print(f"\n📊 Correlation Metrics:")
print(f"   Pearson Correlation:  {pearson_corr:.3f} (p-value: {pearson_p:.3f})")
print(f"   Spearman Correlation: {spearman_corr:.3f} (p-value: {spearman_p:.3f})")

print(f"\n✅ Agreement Metrics:")
print(f"   Exact Agreement:      {exact_agreement}/{len(df_relevance)} ({exact_agreement/len(df_relevance)*100:.1f}%)")
print(f"   Within 1 point:        {within_one}/{len(df_relevance)} ({within_one/len(df_relevance)*100:.1f}%)")
print(f"   Mean Absolute Error:   {mean_absolute_error:.2f}")
print(f"   Max Difference:        {max_difference} point(s)")

print(f"\n📈 Score Statistics:")
print(f"   Human Relevance - Mean: {np.mean(human_relevance):.2f}, Std: {np.std(human_relevance):.2f}")
print(f"   AI Relevance - Mean:    {np.mean(ai_relevance):.2f}, Std: {np.std(ai_relevance):.2f}")

print("\n" + "="*80)
print("DETAILED COMPARISON TABLE")
print("="*80)
display_df = df_relevance[["Summary_ID", "Human_Relevance", "AI_Relevance", "Absolute_Difference"]].copy()
display_df["Agreement_Status"] = display_df["Absolute_Difference"].apply(
    lambda x: "✅ Exact" if x == 0 else "⚠️ Close" if x == 1 else "❌ Disagree"
)
print(display_df.to_string(index=False))

# Identify cases of disagreement
disagreements = df_relevance[df_relevance["Absolute_Difference"] > 0]
if len(disagreements) > 0:
    print("\n" + "="*80)
    print("CASES WHERE HUMAN AND AI DISAGREE")
    print("="*80)
    for idx, row in disagreements.iterrows():
        print(f"\n📝 Summary {int(row['Summary_ID'])}:")
        print(f"   Human Score: {int(row['Human_Relevance'])} | AI Score: {int(row['AI_Relevance'])} | Difference: {int(row['Absolute_Difference'])}")
        print(f"   Generated: {row['Generated_Summary'][:100]}...")
        print(f"   Reference: {row['Reference_Summary']}")


Correlation between human and AI relevance scores: nan
   Human_Relevance  AI_Relevance  Difference
0                5             5           0
1                5             5           0
2                5             5           0
3                5             5           0
4                5             5           0


In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Human vs AI Relevance Score Comparison', fontsize=16, fontweight='bold', y=0.995)

# 1. Scatter plot with correlation line
ax1 = axes[0, 0]
ax1.scatter(human_relevance, ai_relevance, s=200, alpha=0.7, c=df_relevance['Absolute_Difference'], 
            cmap='RdYlGn_r', edgecolors='black', linewidth=2)
ax1.plot([1, 5], [1, 5], 'r--', alpha=0.5, label='Perfect Agreement')
ax1.set_xlabel('Human Relevance Score', fontsize=12, fontweight='bold')
ax1.set_ylabel('AI Relevance Score', fontsize=12, fontweight='bold')
ax1.set_title(f'Scatter Plot (Pearson r={pearson_corr:.3f})', fontsize=13, fontweight='bold')
ax1.set_xlim(0.5, 5.5)
ax1.set_ylim(0.5, 5.5)
ax1.set_xticks(range(1, 6))
ax1.set_yticks(range(1, 6))
ax1.grid(True, alpha=0.3)
ax1.legend()
cbar1 = plt.colorbar(ax1.collections[0], ax=ax1)
cbar1.set_label('Absolute Difference', fontsize=10)

# Add annotations for each point
for idx, row in df_relevance.iterrows():
    ax1.annotate(f"S{int(row['Summary_ID'])}", 
                (row['Human_Relevance'], row['AI_Relevance']),
                xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')

# 2. Side-by-side bar chart
ax2 = axes[0, 1]
x = np.arange(len(df_relevance))
width = 0.35
ax2.bar(x - width/2, df_relevance['Human_Relevance'], width, label='Human', 
        color='#2ecc71', alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.bar(x + width/2, df_relevance['AI_Relevance'], width, label='AI Judge', 
        color='#3498db', alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_xlabel('Summary ID', fontsize=12, fontweight='bold')
ax2.set_ylabel('Relevance Score', fontsize=12, fontweight='bold')
ax2.set_title('Score Comparison by Summary', fontsize=13, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([f"S{i+1}" for i in range(len(df_relevance))])
ax2.set_ylim(0, 5.5)
ax2.set_yticks(range(1, 6))
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (h, a) in enumerate(zip(df_relevance['Human_Relevance'], df_relevance['AI_Relevance'])):
    ax2.text(i - width/2, h + 0.1, f'{int(h)}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    ax2.text(i + width/2, a + 0.1, f'{int(a)}', ha='center', va='bottom', fontweight='bold', fontsize=10)

# 3. Difference distribution
ax3 = axes[1, 0]
colors = ['#2ecc71' if d == 0 else '#f39c12' if d == 1 else '#e74c3c' for d in df_relevance['Absolute_Difference']]
bars = ax3.bar(range(1, len(df_relevance)+1), df_relevance['Absolute_Difference'], 
               color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax3.set_xlabel('Summary ID', fontsize=12, fontweight='bold')
ax3.set_ylabel('Absolute Difference', fontsize=12, fontweight='bold')
ax3.set_title('Absolute Difference Between Human and AI Scores', fontsize=13, fontweight='bold')
ax3.set_xticks(range(1, len(df_relevance)+1))
ax3.set_xticklabels([f"S{i+1}" for i in range(len(df_relevance))])
ax3.set_ylim(-0.2, max(df_relevance['Absolute_Difference']) + 0.5)
ax3.grid(True, alpha=0.3, axis='y')
ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, df_relevance['Absolute_Difference'])):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{int(val)}', ha='center', va='bottom', fontweight='bold', fontsize=11)

# 4. Agreement summary pie chart
ax4 = axes[1, 1]
agreement_counts = [
    (df_relevance['Absolute_Difference'] == 0).sum(),
    ((df_relevance['Absolute_Difference'] == 1)).sum(),
    (df_relevance['Absolute_Difference'] > 1).sum()
]
labels = ['Exact Match\n(0 difference)', 'Close Match\n(1 difference)', 'Disagree\n(>1 difference)']
colors_pie = ['#2ecc71', '#f39c12', '#e74c3c']
explode = (0.05, 0.05, 0.1) if agreement_counts[2] > 0 else (0.05, 0.05, 0)

wedges, texts, autotexts = ax4.pie(agreement_counts, labels=labels, colors=colors_pie, 
                                    autopct='%1.1f%%', explode=explode, startangle=90,
                                    textprops={'fontsize': 11, 'fontweight': 'bold'})
ax4.set_title('Agreement Distribution', fontsize=13, fontweight='bold')

# Make percentage text larger and bold
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(12)
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)
print(f"""
1. 📊 Correlation Analysis:
   - {'Strong positive correlation' if pearson_corr > 0.7 else 'Moderate correlation' if pearson_corr > 0.4 else 'Weak correlation'} 
     between human and AI scores (r={pearson_corr:.3f})
   - This suggests the AI judge {'closely' if pearson_corr > 0.7 else 'somewhat'} aligns with human judgment

2. ✅ Agreement Patterns:
   - {exact_agreement}/{len(df_relevance)} summaries have exact agreement ({exact_agreement/len(df_relevance)*100:.1f}%)
   - {within_one}/{len(df_relevance)} summaries are within 1 point ({within_one/len(df_relevance)*100:.1f}%)
   - Mean absolute error of {mean_absolute_error:.2f} indicates {'excellent' if mean_absolute_error < 0.5 else 'good' if mean_absolute_error < 1.0 else 'moderate'} alignment

3. 🔍 When They Disagree:
   - {'No major disagreements found' if max_difference <= 1 else f'Maximum difference of {max_difference} points observed'}
   - Disagreements often occur when summaries are {'too verbose' if any(df_relevance['Absolute_Difference'] > 0) else 'well-matched'} 
     or miss subtle nuances that humans catch

4. 💡 Practical Implications:
   - AI-as-a-judge can be {'highly reliable' if pearson_corr > 0.7 and mean_absolute_error < 0.5 else 'moderately reliable'} 
     for automated evaluation
   - Human review is still valuable for edge cases and nuanced quality assessment
   - Combining both approaches provides the most robust evaluation pipeline
""")